Установка нужных библиотек

In [ ]:
pip install psycopg2-binary
pip install pandas
pip install numpy
pip install matplotlib

Импорт библиотек


In [1]:
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import psycopg2
import time
import seaborn as sns # seaborn - надстройка над pyplot (она необязателена)

import it4fin_functions as it4fin # мои функции выделенные в отдельный модуль

In [ ]:
# настройка для pandas для отображения всех строк и колонок
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

sns.set() # делаем графики более красивыми через seaborn


Создание соединения с бд

In [3]:
cnxn = psycopg2.connect(user='postgres',
                        database='postgres',
                        host='localhost',
                        port='5432',
                        password='12345')
print(cnxn.get_dsn_parameters())
# проверочный запрос
pd.read_sql_query("SELECT * FROM stock_orders LIMIT 1", cnxn)

{'user': 'postgres', 'channel_binding': 'prefer', 'dbname': 'postgres', 'host': 'localhost', 'port': '5432', 'options': '', 'sslmode': 'prefer', 'sslcompression': '0', 'sslsni': '1', 'ssl_min_protocol_version': 'TLSv1.2', 'gssencmode': 'disable', 'krbsrvname': 'postgres', 'target_session_attrs': 'any'}


,no,security_code,buysell,order_no,action,price,volume,trade_no,trade_price,order_time
0,3672830,TATN,B,1806907,1,348.25,5930,None,None,11:20:05.275000


In [ ]:
def get_spread_series(cnxn, security_code, end_time, start_time='0:00:00', freq=0):
    '''
    Основная фцнкция для выполнения дз

    :param cnxn: соединение с базой данных
    :param security_code: тикер для ценной бумаги, например, 'AFLT'
    :param start_time: (optional) начало временного интервала, в формате 'HH:MI:SS.mss'
    :param end_time: конец временного интервала в том же формате
    :param freq: (optional), частота в миллисекундах

    :returns: `DataFrame`, определяющий биржевой стакан для ценной бумаги (buy volume - price - sell volume) и бид аск спред как `int` в слyчае заданы только `end_time` и `security_code` . Если, в добавок, даны `start_time` и `freq` то так же возвращает `Series` бид-аск спредов для временного интервала между `start_time` и `end_time`

    '''

    # считываем шаблон запроса для получения таблицы
    with open('template.sql', 'r', encoding='utf-8') as f:
        query_template = f.read()



    print(query_template)

    pass

In [ ]:
get_spread_series(cnxn, security_code='YNDX', end_time='12:00:00')

In [ ]:
# читаем шаблон для получения таблицы
with open('template.sql', 'r', encoding='utf-8') as f:
    query_template = f.read()

In [5]:
security_code = 'YNDX'
start_time = '12:00:00' # default
end_time = '14:00:00'
freq = 0 # default


# обработка дефолтного случая задания функции
if start_time == '0:00:00' and freq == 0:
    query = query_template.format(security_code=security_code,
                                  time1=start_time,
                                  time2=end_time)
else: # иначе сначала обработаем таблицу до start_time а потом начнется самое интересное
    query = query_template.format(security_code=security_code,
                                  time1='0:00:00',
                                  time2=start_time)

# print(query)

raw = pd.read_sql_query(query, cnxn)
raw

,buysell,order_no,price,remaining_volume
0,B,192,1088.8,0.0
1,B,1472,1050.1,0.0
2,B,1473,1050.1,5.0
3,B,1491,1066.6,0.0
4,B,1674,1090.0,300.0
...,...,...,...,...
3864,S,2503055,1133.9,0.0
3865,S,2505181,1190.3,2.0
3866,S,2508530,1133.5,25.0
3867,S,2509525,1130.0,0.0


In [9]:
dom, bid, ask = it4fin.get_dom_from_raw(raw)
dom
# dom.loc[1000.0] - получить значение по индексу

,buy,sell
price,,
1222.9,0.0,100.0
1195.0,0.0,1.0
1190.3,0.0,2.0
1189.0,0.0,2.0
1160.0,0.0,50.0
1159.3,0.0,1.0
1152.7,0.0,2.0
1150.5,0.0,1.0
1150.0,0.0,1039.0


In [11]:
bid, ask, (ask - bid) * 2 / (ask + bid)

(1130.7, 1133.5, 0.002473279745605472)

In [ ]:
security_code = 'YNDX'
end_time = '12:00:00'
start_time = '12:01:00' # default
freq = 0 # default


# обработка дефолтного случая задания функции
if start_time == '0:00:00' and freq == 0:
    query = query_template.format(security_code=security_code,
                                  time1=start_time,
                                  time2=end_time)
else: # иначе сначала обработаем таблицу до start_time а потом начнется самое интересное
    query = query_template.format(security_code=security_code,
                                  time1='0:00:00',
                                  time2=start_time)

# print(query)

raw = pd.read_sql_query(query, cnxn)
raw


In [67]:
security_code = 'YNDX'
start_time = '12:00:00' # default
end_time = '12:01:00.01'
freq = 1000 # default

from datetime import datetime, timedelta

# обработка времени формата час:минута:секунда или час:минута:секунда.миллисукенда
if '.' in start_time:
    stime = datetime.strptime(start_time, '%H:%M:%S.%f')
else:
    stime = datetime.strptime(start_time, '%H:%M:%S')

#
if '.' in end_time:
    etime = datetime.strptime(end_time, '%H:%M:%S.%f')
else:
    etime = datetime.strptime(end_time, '%H:%M:%S')

stime, etime

(datetime.datetime(1900, 1, 1, 12, 0),
 datetime.datetime(1900, 1, 1, 12, 1, 0, 10000))

In [68]:
# цикл с постусловием
while True:
    time1 = stime
    time2 = stime + timedelta(milliseconds=freq)

    print(time1.strftime('%H:%M:%S.%f'), time2.strftime('%H:%M:%S.%f'))

    # постусловие
    if time2 >= etime - timedelta(milliseconds=freq):
        break

    stime = time2



12:00:00.000000 12:00:01.000000
12:00:01.000000 12:00:02.000000
12:00:02.000000 12:00:03.000000
12:00:03.000000 12:00:04.000000
12:00:04.000000 12:00:05.000000
12:00:05.000000 12:00:06.000000
12:00:06.000000 12:00:07.000000
12:00:07.000000 12:00:08.000000
12:00:08.000000 12:00:09.000000
12:00:09.000000 12:00:10.000000
12:00:10.000000 12:00:11.000000
12:00:11.000000 12:00:12.000000
12:00:12.000000 12:00:13.000000
12:00:13.000000 12:00:14.000000
12:00:14.000000 12:00:15.000000
12:00:15.000000 12:00:16.000000
12:00:16.000000 12:00:17.000000
12:00:17.000000 12:00:18.000000
12:00:18.000000 12:00:19.000000
12:00:19.000000 12:00:20.000000
12:00:20.000000 12:00:21.000000
12:00:21.000000 12:00:22.000000
12:00:22.000000 12:00:23.000000
12:00:23.000000 12:00:24.000000
12:00:24.000000 12:00:25.000000
12:00:25.000000 12:00:26.000000
12:00:26.000000 12:00:27.000000
12:00:27.000000 12:00:28.000000
12:00:28.000000 12:00:29.000000
12:00:29.000000 12:00:30.000000
12:00:30.000000 12:00:31.000000
12:00:31